In [455]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [456]:
train = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Processed/2010-2014/train/officer_profile.csv", low_memory=False)
test = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Boston_College_Courses/Thesis/Data/Processed/Testing/officer_profile.csv", low_memory=False)

In [457]:
train = train.drop(columns = ['OfficerFirst', 'OfficerLast', 'ApptDate'])
test = test.drop(columns = ['OfficerFirst', 'OfficerLast', 'ApptDate'])

In [458]:
train.columns, test.columns

(Index(['OfficerID', 'Gender', 'Race', 'Rank', 'Age', 'Beat', 'historic_counts',
        'prev_allegations', 'new_allegations', 'ApptYear', 'YearsInForce'],
       dtype='object'),
 Index(['OfficerID', 'Gender', 'Race', 'Rank', 'Age', 'Beat', 'historic_counts',
        'prev_allegations', 'new_allegations', 'ApptYear', 'YearsInForce'],
       dtype='object'))

In [459]:
# if beat is nan, remove row
print(train.shape)
train = train.dropna(subset=['Beat'])
print(train.shape)

(50218, 11)
(14021, 11)


In [460]:
print(test.shape)
test = test.dropna(subset=['Beat'])
print(test.shape)

(50218, 11)
(15380, 11)


In [461]:
# if there are nan values, drop the row
train = train.dropna()
test = test.dropna()

In [462]:
train["Beat"] = train["Beat"].astype(int)
test["Beat"] = test["Beat"].astype(int)

In [463]:
train.columns, test.columns

(Index(['OfficerID', 'Gender', 'Race', 'Rank', 'Age', 'Beat', 'historic_counts',
        'prev_allegations', 'new_allegations', 'ApptYear', 'YearsInForce'],
       dtype='object'),
 Index(['OfficerID', 'Gender', 'Race', 'Rank', 'Age', 'Beat', 'historic_counts',
        'prev_allegations', 'new_allegations', 'ApptYear', 'YearsInForce'],
       dtype='object'))

In [464]:
print(train.head())

   OfficerID Gender           Race             Rank  Age  Beat  \
0          4      M          White  Po As Detective   80   321   
1         13      M          White   Police Officer   69  1511   
2         17      M          Black   Police Officer   64  2525   
3         19      M  Asian/Pacific   Police Officer   73  1122   
4         34      M          Black   Police Officer   49  1212   

   historic_counts  prev_allegations  new_allegations  ApptYear  YearsInForce  
0               71                 2                0    1969.0          41.0  
1               75                 2                1    1982.0          28.0  
2              260                25                0    1989.0          21.0  
3                2                 4                1    1999.0          11.0  
4                3                11                1    1998.0          12.0  


In [465]:
# change column name of new_allegation to new_allegations
test = test.rename(columns={"new_allegation": "new_allegations"})

In [466]:
# new_allegation is target
X_train = train.drop(columns = ['new_allegations'])
y_train = train['new_allegations']
X_test = test.drop(columns = ['new_allegations'])
y_test = test['new_allegations']

In [467]:
model = CatBoostRegressor(iterations=2, depth=2, learning_rate=1, loss_function='RMSE', cat_features=['Gender', 'Race', 'Beat', "Rank"])

In [468]:
model.fit(X_train, y_train)

0:	learn: 2.3755760	total: 4.81ms	remaining: 4.81ms
1:	learn: 2.3183029	total: 7.22ms	remaining: 0us


In [469]:
model.get_feature_importance()

array([ 1.96469004,  0.        ,  0.        ,  0.        , 18.05545158,
        0.        ,  0.        , 79.97985838,  0.        ,  0.        ])

In [470]:
# get predicited results, compare to actual results
y_pred = model.predict(X_test)
# round to nearest integer
# y_pred = np.round(y_pred)
mean_absolute_error(y_test, y_pred)

1.5206550021558927

In [471]:
# make y_pred and y_test into a dataframe
predictions = pd.DataFrame(y_pred)
actual = pd.DataFrame(y_test)
# combine y_pred and y_test into one dataframe
predictions = pd.concat([predictions, actual], axis=1)
predictions.columns = ['Predicted', 'Actual']
print(y_pred.shape)

(15269,)


In [472]:
# drop NaN values
predictions = predictions.dropna()

In [473]:
# split data into 5 bins
predictions['Predicted'] = pd.qcut(predictions['Predicted'], 5, labels=False, duplicates = 'drop')
print(predictions['Predicted'].value_counts())
# calculate mean of actual values in each bin
actual_mean = predictions.groupby('Predicted')['Actual'].mean()
actual_mean

1    6470
2    4520
0    3533
3     196
Name: Predicted, dtype: int64


Predicted
0    0.321540
1    0.336631
2    0.335398
3    0.234694
Name: Actual, dtype: float64

In [474]:
# get mean squared error of this top 1%
mean_absolute_error(top_1['Actual'], top_1['Predicted'])

8.571428571428571